Работа №3: Исследование алгоритмов классификации

0) Оценка датасета:
    1) Оценить какой атрибут выступит в роли целевого класса (метки)
    2) Оценить сбалансированность классов

Я решила взять новый датасет:
https://www.kaggle.com/datasets/muhammadbinimran/housing-price-prediction-data/ 
Это датасет о качестве жилья, он включает в себя площадь, спальни, ванные комнаты, типы районов, год постройки и цену.

In [70]:
import numpy as np
import pandas as pd
df = pd.read_csv('housing_price_dataset.csv')

def category(row):
    first = row["Price"]

    if (first < 18.5):
        return 'LowPrice'
    elif (first> 180000 and  first < 250000):
        return 'Normal'
    else:
        return 'HardLuxe'
        

df["Category"] = df.apply(lambda row: category(row), axis=1)
df.head()


,SquareFeet,Bedrooms,Bathrooms,Neighborhood,YearBuilt,Price,Category
0,2126,4,1,Rural,1969,215355.283618,Normal
1,2459,3,2,Rural,1980,195014.221626,Normal
2,1860,2,1,Suburb,1970,306891.012076,HardLuxe
3,2294,2,1,Urban,1996,206786.787153,Normal
4,2130,5,2,Suburb,2001,272436.239065,HardLuxe


Здесь целевым классом будет "Category". Теперь проверим на сбалансированность.

In [43]:
from collections import Counter

category_count = Counter(df["Category"])

imbalance = 1 - sum(category_count.values()) / len(df)
print ("LowPrice: " , category_count['HardLuxe'])
print ("Normal: " , category_count['Normal'])
print ("HardLuxe: " ,category_count['LowPrice'])
print("Imbalance:", imbalance)

LowPrice:  19077
Normal:  16302
HardLuxe:  14621
Imbalance: 0.0


Видим, что класс сбалансирован.

1. Необходимо оценить и сравнить результаты классификации, используя следующие
алгоритмы классификации:
* kNN
* дерево решений

1.1) kNN

In [71]:
#Удалим класс Neighborhood он не подходит нам по типу 
df.drop('Neighborhood', axis= 1 , inplace= True ) 

In [58]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
%matplotlib inline

#Стандартизация датасета
scaler = StandardScaler()
scaler.fit(df.drop('Category', axis=1))
# применим метод transform для стандартизации всех признаков,
scaled_features = scaler.transform(df.drop('Category', axis=1))
scaled_data = pd.DataFrame(scaled_features, columns = df.drop('Category', axis=1).columns)

#Разделение датасета на обучающие и тестовые данные
x = scaled_data
y = df['Category']
#будем использовать test_size 20%
x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(x, y, test_size = 0.2)

#Обучение модели K-ближайших соседей
model = KNeighborsClassifier(n_neighbors = 1)
model.fit(x_training_data, y_training_data)

#Делаем предсказания с помощью алгоритма K-ближайших соседей
predictions = model.predict(x_test_data)
print(classification_report(y_test_data, predictions))

              precision    recall  f1-score   support

    HardLuxe       0.98      0.97      0.97      3834
    LowPrice       0.97      0.97      0.97      2911
      Normal       0.94      0.94      0.94      3255

    accuracy                           0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000



In [59]:
#матрицa ошибок
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test_data, y_pred)) 

[[1498 1106 1230]
 [1148  817  946]
 [1244  993 1018]]


Вывод:
Алгоритм k-ближайших соседей (kNN) показывает хорошую точность классификации. 
Первый класс имеет 1498 ошибок, что является самым высоким значением среди всех классов. Это может быть связано с тем, что этот класс является самым многочисленным и алгоритм может испытывать трудности в его классификации.
Для улучшения точности алгоритма можно попробовать использовать различные метрики расстояния при вычислении близости между точками. Некоторые метрики могут быть более эффективны для определенных типов данных.

1.2) Дерево решений

In [62]:
from sklearn.tree import DecisionTreeClassifier
#Разделение датасета на обучающие и тестовые данные
#будем использовать test_size 20%
x = scaled_data
y = df['Category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

#Алгоритм дерева решений
classifier = DecisionTreeClassifier() 
classifier.fit(X_train, y_train)
#Делаем предсказания с помощью алгоритма K-ближайших соседей
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))
#матрицa ошибок
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

    HardLuxe       1.00      1.00      1.00      3772
    LowPrice       1.00      1.00      1.00      2907
      Normal       1.00      1.00      1.00      3321

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000

[[3772    0    0]
 [   0 2907    0]
 [   0    0 3321]]


Обший вывод:
Результаты алгоритмов отличаются несильно. 

2. Сравнить полученные результаты с помощью различных метрик оценки качества:
Accuracy
Presicion, Recall, F-measure
ROC

Точность (Accuracy) - это процент правильно классифицированных данных. В моём случае точность для KNN составляет 96%, а для Дерева решений - 100%.

Precision - это отношение верно классифицированных объектов к числу объектов, классифицированных как положительные. 

Recall - это отношение верно классифицированных положительных объектов ко всем положительным объектам. 

F1-measure - это комбинация Precision и Recall. 

ROC-кривая (Receiver Operating Characteristic) показывает зависимость между долей верно классифицированных положительных образцов (TPR) и долей неверно классифицированных отрицательных образцов (FPR).

По результатам двух алгоВ результате сравнения алгоритмов KNN и Дерева решений с использованием различных метрик качества, таких как точность (Accuracy), Precision, Recall, F1-score и ROC-кривые, было выявлено, что оба алгоритма достаточно хорошо справляются с задачей классификации. Однако алгоритм Дерева решений показал немного лучшие результаты, возможно благодаря своей способности обрабатывать нелинейные зависимости между признаками.ритмов мы видим, что дерево решений точнее, чем алгоритм kNN.

3. Объяснить полученные результаты

Результаты могут варьироваться в зависимости от набора данных и гиперпараметров, выбранных для каждого алгоритма. Поэтому, чтобы получить более точные результаты, можно провести дополнительные эксперименты с различными параметрами и наборами данных.